In [12]:
import cv2 
import numpy as np

In [13]:
image = cv2.imread("test_image.JPG", cv2.IMREAD_GRAYSCALE)
cap = cv2.VideoCapture(0)

In [14]:
#Feature detection Algorithm - SIFT
sift = cv2.xfeatures2d.SIFT_create()
kp_image, desc_image = sift.detectAndCompute(image, None)

In [15]:
#Feature matching
index_params = dict(algorithm=0, trees=5)
search_params = dict()
#Load Algorithm for feature matching - FLANN
flann = cv2.FlannBasedMatcher(index_params, search_params)

In [16]:
#Detect features from Webcam and compare
_, frame = cap.read()
grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
kp_grayframe, desc_grayframe = sift.detectAndCompute(grayframe, None)
matches = flann.knnMatch(desc_image, desc_grayframe, k=2)
good_points = []
for m, n in matches:
    if m.distance < 0.6 * n.distance:
        good_points.append(m)

In [17]:
#find homography
query_pts = np.float32([kp_image[m.queryIdx].pt for m in good_points]).reshape(-1, 1, 2)
train_pts = np.float32([kp_grayframe[m.trainIdx].pt for m in good_points]).reshape(-1, 1, 2)
matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0)
matches_mask = mask.ravel().tolist()

In [18]:
#perspective transform
h, w = image.shape
pts = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
dst = cv2.perspectiveTransform(pts, matrix)

In [19]:
#display result
homography = cv2.polylines(frame, [np.int32(dst)], True, (255, 0, 0), 3)
cv2.imwrite("saved.jpg", homography)

True